<h2><center>Filtered $P_N$ Method ($FP_N$)</center></h2>

This notebook summarizes filtered $P_N$, a variation to the $P_N$-approximation which was introduced in [this paper](https://arxiv.org/abs/0907.2099) for two-dimensional problems and has been extended to three-dimensional problems in [this paper](https://arxiv.org/pdf/1209.1634.pdf).

The filtered $P_N$ method helps with Gibbs phenomen occuring in $P_N$ simulations with high density gradients. In those cases, the $P_N$ solution produces negative results which go away (or are reduced) with $FP_N$. The following images compares $P_5$ (left) against $FP_5$ (right). The gray areas in the left image are clamped negative results:

<center>
<img src="spherical_harmonics_method_closures_FPN/p5.png" width="450" style="display:inline;margin:1px">
<img src="spherical_harmonics_method_closures_FPN/fp5.png" width="450" style="display:inline;margin:1px">
</center>

The core idea behind $FP_N$ is to introduce an additional scattering term with a highly forward peaked phase function $f$. Since it only depends on the scattering angle, it can be understood as a filter which is applied to $L$. The general idea has some similarities to mollification methods in computer graphics.


### Filtered SH expansion

Consider the standard SH expansion of the radiance field:

$$
L_N = \sum_{l=0}^{N}\sum_{m=-l}^{l}L^{lm}Y^{lm}
$$

The SH projection finds coefficients $L^{lm}$, such that the mean square error between the radiance field and its truncated expansion $L_N$ is minimized. However, this minimization does not take the gradient of $L$ (in angular domain) into account (see chapter 18.20.3 in [this book](http://perso.uclouvain.be/alphonse.magnus/num1a/boyd01.pdf)). This gradient can be significant due to shadows etc. Therefore, the expansion can have high oscillations in angular domain represented by high values for higher order coefficients. These oscillations will be advected by $P_N$ through space.

The idea behind $FP_N$ is, to apply a filter to the radiance field, which smoothens out higher angular frequencies. This is done by applying a convolution with a filter function. We know that a convolution can be expressed as a dot product between SH coefficients of the filter function and the radiance field.

Therefore, in the most general form, the filtered SH expansion $\hat L$ can be expressed as a scaling of the radiance field SH coefficients:

$$
\hat L_N = \sum_{l=0}^{N}\sum_{m=-l}^{l}f\left(\frac{l}{N+1}\right)^{\sigma_f\Delta t}L^{lm}Y^{lm}
$$

$f$ is a continuous function (the filter) which takes the normalized SH band as input. The exponent $\sigma_f$ has been introduced to allow tuning of the filter strength by the user. The time step $\Delta t$ in the exponent is there, to allow the formal derivation of the $FP_N$ equation in continuum form by letting $t\rightarrow 0$.










### The $FP_N$ equations

In order to derive the $FP_N$ equations, the filtered expansion $\hat L_N$ is substituted for $L$ in the isotropic RTE, which is then projected into SH (by multiplying with $\overline{Y^{lm}}$ and integrating over solid angle). Letting $\Delta t \rightarrow 0$, we arrive at the final $FP_N$ equations (see p.12 [here](http://www.sciencedirect.com/science/article/pii/S0021999113001125) for details):

$$
\partial_t\vec{u} + M_x\cdot\partial_x\vec{u}+M_y\cdot\partial_y\vec{u}+M_z\cdot\partial_z\vec{u} + C\cdot\vec{u} + \sigma_f F\cdot\vec{u} = \vec{q}
$$

where $\vec{u}$ is the solution vector containing the spatially varying SH coefficients $L^{lm}$ for which we want to solve.

The new term $ \sigma_f F\cdot\vec{u}$ is similar to a scattering term. The filtering cross-section $\sigma_f$ is a user specified parameter, which drives the strength of the filter. A value of zero retrieves the standard $P_N$ method. A physical interpretation of that parameter would be the average distance a particle travels before it interacts with a *homogeneous medium which smoothens the radiance field in angular domain through scattering*.

The matrix $F$ is a diagonal matrix with:

$$
F_{ll} = \operatorname{log}f\left(\frac{l}{N+1}\right)
$$

For the derivation of $F$ see p.12 Eq.37 in [this paper](http://www.sciencedirect.com/science/article/pii/S0021999113001125).


### Solving $FP_N$ in starmap

Solving $FP_N$ equations e.g. in starmap seems to be very straight forward and has been done [in this paper](https://www.ki-net.umd.edu/pubs/files/main.pdf).

Due to the $\sigma_f$ factor, the additional scattering term can not be absorbed into the matrix $C$. The filtering term is accounted for by applying $\sigma_f F$ to $\vec{u}$ after every substep during time stepping. The filter used by the referenced paper is:

$$
f(\eta) = \operatorname{exp}( c\eta^\alpha ) \qquad \text{with}\; c=\operatorname{log}(\epsilon_M)
$$

where $\epsilon_M$ is the machine precision. The parameter $\alpha$ is another user defined parameter, which allows to tune the broadness of the effect, the filter has on higher order coefficients vs. lower order.

#### Notes

Another closure which addresses the problem of negative values in the solution can be found [here](https://www.ki-net.umd.edu/pubs/files/FRG-2009-Hauck-Cory.ppn2.pdf).